In [183]:
from nltk.util import pr
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re
import nltk

In [180]:
import pandas as pd

In [181]:
data=pd.read_csv('\\speech\\labeled_data.csv')

In [182]:
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [184]:
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [185]:
data["labels"] = data["class"].map({0: "Hate Speech", 
                                    1: "Offensive Language", 
                                    2: "No Hate and Offensive"})

In [186]:
data.head(10)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,labels
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Language
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Language
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Language
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Language
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",Offensive Language
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ...",Offensive Language
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,Offensive Language
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ...",Offensive Language
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria...",Offensive Language


In [193]:
category = data['labels'].value_counts().reset_index()
category

,index,labels
0,Offensive Language,19190
1,No Hate and Offensive,4163
2,Hate Speech,1430


In [194]:
df= data[["tweet", "labels"]]
df.head()

,tweet,labels
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Language
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Language
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Language
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Language


In [195]:
df['tweet'][0]

"!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out..."

In [196]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

In [ ]:
df["new_tweet"] = df["tweet"].apply(clean)

In [198]:
df.head()

,tweet,labels,new_tweet
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive,rt mayasolov woman shouldnt complain clean ho...
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Language,rt boy dat coldtyga dwn bad cuffin dat hoe ...
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Language,rt urkindofbrand dawg rt ever fuck bitch sta...
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Language,rt cganderson vivabas look like tranni
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Language,rt shenikarobert shit hear might true might f...


In [202]:
x = np.array(df["new_tweet"])
y = np.array(df["labels"])
cv = CountVectorizer()
X = cv.fit_transform(x)
print(x.shape)

(24783,)


In [206]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [208]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)

In [209]:
clf.fit(X_train,y_train)

RandomForestClassifier(random_state=42)

In [210]:
accuracy = clf.score(X_test, y_test)
print("Précision du modèle : {:.2f}%".format(accuracy * 100))

Précision du modèle : 89.02%


In [216]:
sample = X_test[0]
sample_label=y_test[0]
print(f"predction:{clf.predict(sample)[0]}   True label: {sample_label}")

predction:Offensive Language   True label: Offensive Language
